# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 11: Limpieza de Datos</center></h1>

Este ejercicio busca que los estudiantes aprendan a usar algunas de las librearías de Python y Pandas para limpiar datos crudos de un repositorio.

## 1. El problema con los datos

Volvamos un minuto a revisar los datos de los taxis en NYC y tratar de sacar algunas conclusiones. 

Supongamos que nuestro objetivo es ver cuantos viajes se hacen en la ciudad usando los taxis amarillos para un mes en particular, cosa de determinar si necesitamos más o menos taxis en algunas zonas de la ciudad.

In [ ]:
import pyarrow.parquet as pq

# leer la base parquet
trips = pq.read_table('data\yellow_tripdata_2022-01.parquet')
# transformar a dataframe de pandas
trips = trips.to_pandas()
trips.info()

Ahora que tenemos los datos en un DataFrame, podemos rápidamente saber la cantidad de viajes por zona agrupando como lo hicimos antes y contando la cantidad de registros.

In [ ]:
trips_by_loc = trips[["PULocationID"]].groupby("PULocationID").size()
trips_by_loc.sort_values()

Con este simple cálculo podemos concluir que las zonas 237, 236, 132, 161 y 186 son las más usadas para tomar taxi, mientras las zonas 176, 27, 172, 105 y 187 son las menos usadas en ese mes. Pero, ¿está correcto lo que hicimos?

In [ ]:
trips[trips["PULocationID"] == 187]

In [ ]:
trips[trips["PULocationID"] == 237]

In [ ]:
trips[(trips["PULocationID"] == 237) & (trips["passenger_count"] >= 1)]

In [ ]:
trips["trip_distance"].sort_values()

Claramente tenemos un gran número de registros con problemas y por ende no podemos confiar en el resumen que hicismos antes para definir algo. Necesitamos limpiar los datos que tienen problemas.

In [ ]:
viajes_limpios = trips[(trips["passenger_count"] >= 1) & (trips["trip_distance"] > 0) & (trips["tpep_pickup_datetime"].dt.month == 1)]
len(viajes_limpios)

In [ ]:
trips_by_loc = viajes_limpios[["PULocationID"]].groupby("PULocationID").size()
trips_by_loc.sort_values()

## 2. Ejemplo de limpieza de Datos.

El set de datos que utilizaremos en este ejercicio proviene de la página `insideairbnb.com`, un grupo de lobby anti-Airbnb que hace scraping de las propiedades publicadas en distintas ciudades del mundo, para estudiar el impacto que esta plataforma tiene sobre los barrios y el desarrollo urbano. En este caso, procesaremos el dataset de propiedades disponibles en Santiago de Chile (actualizado al 28 de junio de 2023):

http://insideairbnb.com/index.html

In [ ]:
# lectura de datos
import pandas as pd

df0 = pd.read_csv('data\listings.csv')
df0.head()

## 2. Cambios en columnas

Revisemos cómo trabajar sobre las columnas del DataFrame

In [ ]:
df0.columns

### 2.1 Seleccionando algunas columnas

Podemos generar filtros que nos permiten seleccionar las columnas de interés para el trabajo posterior.

In [ ]:
# columnas a dejar
keep_cols = ['id','name','host_id','neighbourhood_group_cleansed','neighbourhood', 'minimum_nights', 'latitude','longitude','price','number_of_reviews']
# nuevo dataframe con las columnas seleccionadas
df = df0[keep_cols]
df.head(10)

### 2.2 Eliminando columnas

En el dataframe anterior notamos que la columna `neighbourhood_group_cleansed` no nos enterga información, por lo que la podemos eliminar.

In [ ]:
df = df.drop(columns=['neighbourhood_group_cleansed'])
df.head()

### 2.3 Tipos de Datos

Otro métido importante que usaremos la clase próxima nos permite ver el tipo de dato que hay en cada columna.

In [ ]:
df.dtypes

In [ ]:
df['minimum_nights'].describe()

In [ ]:
df['minimum_nights'] = df['minimum_nights'].astype('category')

In [ ]:
df['minimum_nights'].dtype

In [ ]:
df['minimum_nights'].describe()